In [5]:
# import dependencies

import pandas as pd
from pandas import read_csv
import numpy as np

In [6]:
# save datasets to paths

path = 'C:\\Users\\vivit\\Downloads\\data.csv'
path2 = 'C:\\Users\\vivit\\Downloads\\term_def.csv'
path3 = 'C:\\Users\\vivit\\Downloads\\term_abb.csv'


# import datasets into dataframes

data_df = read_csv(path)
term_def_df = read_csv(path2)
term_abb_df = read_csv(path3)

In [7]:
data_df

,id,content
0,321712,Hey 👋 \n\nWe re using our bot:\n\nhttps://t.me...
1,321713,Good stuff \n\nI am surprised I took so long t...
2,321717,you are using a non-official one
3,321718,use the one that uniswap uses: https://thegrap...
4,321719,keep in mind this is a hot subgraph so it can ...
...,...,...
44131,374466,Can find it in many places\nAlso on Santiment:...
44132,374467,"guys, does anyone know if there is an applicat..."
44133,374468,Any Lobsters going to Kyiv Web3 Hackathon Sept...
44134,374469,whats funny is that no one complains about the...


In [75]:
# function to extract keywords/phrases from a given sentence

from keyphrase_vectorizers import KeyphraseCountVectorizer
import bs4 as bs  
import urllib.request  
import re, string
import nltk
from nltk.stem.porter import *

from collections import Counter
  
import spacy 
"""nltk.download('punkt')"""

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize.treebank import TreebankWordTokenizer, TreebankWordDetokenizer

import re, string

import spacy 
sp = spacy.load('en_core_web_sm')
stemmer = PorterStemmer()
all_stop_words = sp.Defaults.stop_words

from rake_nltk import Rake
r = Rake(stopwords = all_stop_words,max_length = 3)

from keybert import KeyBERT
from keyphrase_vectorizers import KeyphraseCountVectorizer
vectorizer = KeyphraseCountVectorizer()
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
kw_model = KeyBERT(model=sentence_model)

def clean(sentence):

    # remove all characters that are not alphanumeric characters (/w) and white space and tab (\s) 
    # and replace them with ""
    sentence = re.sub(r'[^\w\s]','',sentence)

    # remove links
    sentence = re.sub(r'http\S+', '',sentence)
    # lowercase
    sentence = sentence.lower()

    # remove punctuations
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))
    

    return sentence
    

def get_article_text_list(article_url):
    
    scrapped_data = urllib.request.urlopen(article_url)
    article = scrapped_data .read()
    parsed_article = bs.BeautifulSoup(article,'lxml')
    paragraphs = parsed_article.find_all('p')
    article_text_list = []
    for p in paragraphs:  
        article_text_list.append(clean(p.text))
            
    return article_text_list
    
def get_keyW_from_long_text(sentence_list):


    # Init KeyBERT

    # here 'keyphrases' gives a list of list of keywords
    keyphrases = kw_model.extract_keywords(docs=sentence_list, stop_words = 'english', 
                                            use_mmr = "True", diversity = 0.8, top_n = 2)

    
    keywords_list = []
    for keyphrase_list in keyphrases:
        for keyphrase in keyphrase_list:
            if keyphrase not in keywords_list:
                keywords_list.append(keyphrase[0])

    return keywords_list


def get_most_common_words(word_list, number_of_words_to_be_extracted):
    # Pass the split_it list to instance of Counter class.
    
    for word in word_list:
        counter = Counter(word_list)
        most_occur = counter.most_common(number_of_words_to_be_extracted)
        m_o_list = []
    for m_o in most_occur:
        m_o_list.append(m_o[0])
        
    return m_o_list
    
def get_keyW_from_short_text(method,sentence):
    
    if method == "rake":

        r.extract_keywords_from_text(sentence)
        res = r.get_ranked_phrases()[0:3]
        return res
    
    elif method == "spacy_nouns":
        
        sentence = sp(sentence)
        res = sentence.noun_chunks
        keyW_str = []
        for word in res:
            word_tokens = TreebankWordTokenizer().tokenize(word.text)
            # removes stopwords from noun phrases 
            filtered_phrase = [w for w in word_tokens if not w in all_stop_words]
            filtered_phrase = TreebankWordDetokenizer().detokenize(filtered_phrase)
            if filtered_phrase!='':
                keyW_str.append(filtered_phrase)
        
        return keyW_str
        
    else: 
        print(method + " not recognised as a method! Try 'spacy_nouns' or 'rake'")
        return None

     


# function to build vocabulary list
def build_vocab(vocab_list , new_words_list):
    flag = 0
    for new_word in new_words_list:

        if new_word not in vocab_list:
            vocab_list.append(new_word)  
            
    vocab_list.sort()
    return vocab_list


In [9]:
# remove nan containing rows

data_df = data_df.dropna()
data_df = data_df.reset_index(drop=True)


"# extracting keywords form sentences and importing them into another column in the dataframe\nfor i in range(len(data_df['content'])):\n    data_df['keywords'][i] = get_keyW_from_short_text(clean(data_df['content'][i]))"

In [105]:
# clean the dataset

for i in range(len(data_df)):
    data_df['content'][i] = clean(data_df['content'][i])

C:\Users\vivit\anaconda3\envs\StockAI37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [106]:
data_df

,id,content,crytpo_related_keywords
0,321712,hey \n\nwe re using our bot\n\n\n\nand also f...,[topics]
1,321713,good stuff \n\ni am surprised i took so long t...,[]
2,321717,you are using a nonofficial one,[]
3,321718,use the one that uniswap uses,[]
4,321719,keep in mind this is a hot subgraph so it can ...,[hot subgraph]
...,...,...,...
43481,374466,can find it in many places\nalso on santiment ...,
43482,374467,guys does anyone know if there is an applicati...,
43483,374468,any lobsters going to kyiv web3 hackathon sept...,
43484,374469,whats funny is that no one complains about the...,


In [107]:
# building vocabulary from given data

new_words_list_list1 = [term_def_df['terms'].tolist(),term_abb_df['terms'].tolist(), 
              term_abb_df['abbreviations'].tolist()]


In [108]:
# cleaning the above lists

vocab = []
for new_words_list_ in new_words_list_list1:
    for word in new_words_list_:
        vocab.append(clean(word))


In [109]:
len(vocab)

316

In [110]:
# extracting new words and adding to vocab. This pulls 10 most prominent word from a given article.

article_ur1s = ['https://ethereum.org/en/eth/',
                'https://www.geeksforgeeks.org/introduction-to-polygon/',
               'https://www.forbes.com/advisor/in/investing/cryptocurrency/best-cryptocurrency-exchanges/',
               'https://en.wikipedia.org/wiki/Web3'
                ]

for article_url in article_ur1s:
    print(article_url)
    x = get_keyW_from_long_text(get_article_text_list(article_url))
    print(len(x))
    x = get_most_common_words(x,10)
    print(len(x))
    vocab = build_vocab(vocab,x)


https://ethereum.org/en/eth/
73
10
https://www.geeksforgeeks.org/introduction-to-polygon/
36
10
https://www.forbes.com/advisor/in/investing/cryptocurrency/best-cryptocurrency-exchanges/
192
10
https://en.wikipedia.org/wiki/Web3
36
10


In [111]:
# 34 new words were found in the articles and added to the vocabulary

len(vocab)

350

In [112]:
# adding some basic words manually

new_words = ['ether', 'coin', 'swap', 'exchange','mine']
build_vocab(vocab, new_words)
len(vocab)

355

In [113]:
# trying to extract crypto related keywords using spacy's phrase matcher

import spacy
sp = spacy.load('en_core_web_sm')


from spacy.matcher import PhraseMatcher
phrase_matcher = PhraseMatcher(sp.vocab)

patterns = [sp(text) for text in vocab]
phrase_matcher.add('matcher', None, *patterns)



In [114]:
# this matcher is able to detect the word 'token' present in the vocab

sentence = sp(data_df['content'][93])
print(sentence)
matched_phrases = phrase_matcher(sentence)
for match_id, start, end in matched_phrases:
    string_id = sp.vocab.strings[match_id]  
    span = sentence[start:end]                   
    print(span.text)

can they just make a token already please 
token


In [115]:
# But it isn't able detect teh word 'ethermine', even though the words 'ether' and 'mine'
# exist in the vocab

sentence = sp(data_df['content'][43484])
print(sentence)
matched_phrases = phrase_matcher(sentence)
for match_id, start, end in matched_phrases:
    string_id = sp.vocab.strings[match_id]  
    span = sentence[start:end]                   
    print(span.text)

whats funny is that no one complains about the txs being rejected and censored by ethermine and flashbots right now not related to ofac


In [116]:
# class contains embedder tools

import math
from collections import Counter

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

ps = PorterStemmer()
wnl = WordNetLemmatizer()


class wordEmbedder:
    
    def __init__(self, embedder):
        
        self.embedder = embedder

        
    def get_vectors(self, list_1, list_2):
        
        self.x = word_tokenize(list_1) 
        self.y = word_tokenize(list_2) 
        
        if self.embedder == "basic":
            x,y = self.basic_embedder()
        elif self.embedder == "word sensitive":
            x,y = self.word_sensitive_embedder()
        else:
            raise Exception(embedder+" embedder not found!")
        
        return x,y
    
    
    # utility functions
    
    # stemmer for list of words
    def get_stemmed_list(self,list_):
        stemmed_list = []
        for word in list_:
            stemmed_list.append(ps.stem(word))
        return stemmed_list

    # lemmatizer for list of words
    def get_lemmatized_list(self,list_):
        lemmatized_list = []
        for word in list_:
            lemmatized_list.append(ps.wnl(word))
        return lemmatized_list

    def is_substring(self,substring, fullstring):
        is_substring = 0
        if substring == '' or fullstring == '':
            return is_substring
        if substring in fullstring:
            is_substring = 1
        return is_substring

            
    # takes sentences as input
    def basic_embedder(self):
        
        self.x = self.get_stemmed_list(self.x)
        self.y = self.get_stemmed_list(self.y)
        
        counter1 = Counter(self.x)
        counter2= Counter(self.y)
    
        all_items = set(counter1.keys()).union( set(counter2.keys()) )
        vector1 = [counter1[k] for k in all_items]
        vector2 = [counter2[k] for k in all_items]
        vector1 = np.array(vector1)
        vector2 = np.array(vector2)
    
        return vector1, vector2

    
    def word_sensitive_embedder(self):

        # x = vocab word/phrase, y = key-word/phrase  
        # if there is a word in y that contains a word in the voacb
        # for ex, crypto-___, it needs to detected
        
        counter1 = Counter(self.x)
        counter2 = Counter(self.y)

        all_items = set(counter1.keys()).union( set(counter2.keys()) )

        for x_ in self.x:
            vector1 = [self.is_substring(x_,k) for k in all_items]
        vector2 = [counter2[k] for k in all_items]
        vector1 = np.array(vector1)
        vector2 = np.array(vector2)

        return vector1, vector2

In [117]:
# class contains similarity detection tools

class keyMatcher:

    def __init__(self, matcher):
        
        self.matcher = matcher
        
        
    def get_similarity(self, vector_1, vector_2):
        
        self.x = vector_1
        self.y = vector_2
        
        if self.matcher == "jaccard":
            similarity = self.jaccard_similarity()
        elif self.matcher == "cosine":
            similarity = self.cosine_similarity()
        else:
            raise Exception(matcher+" matcher not found!")
        
        return similarity

    def jaccard_similarity(self):
    
        intersection = np.logical_and(self.x, self.y)
        union = np.logical_or(self.x, self.y)
        jaccard_similarity = intersection.sum() / float(union.sum())
        
        return jaccard_similarity
    
    def cosine_similarity(self):
        # Compute the dot product between x and y
        dot_product = np.dot(self.x, self.y)

        # Compute the L2 norms (magnitudes) of x and y
        magnitude_x = np.sqrt(np.sum(self.x**2)) 
        magnitude_y = np.sqrt(np.sum(self.y**2))

        # Compute the cosine similarity
        cosine_similarity = dot_product / (magnitude_x * magnitude_y)

        return cosine_similarity

In [141]:
# decision function to extract crypto related words. It checks for a match between words from the
# vocab and inputted sentence

def get_crypto_related_keys(threshold,keyword_extraction_method,embed_func, sim_func, vocab,sentence):

    crypto_related_keys = []
    keyW_list = get_keyW_from_short_text(keyword_extraction_method,sentence)
    for keyW in keyW_list:
        sim = 0
        for word in vocab:
            x,y = embed_func(word,keyW)
            sim += sim_func(x,y)
        if sim>=threshold:
            crypto_related_keys.append(keyW)
  
    return crypto_related_keys


In [134]:
threshold = 0.7
embed_func = wordEmbedder("word sensitive").get_vectors
sim_func = keyMatcher("cosine").get_similarity
data_df['crytpo_related_keywords'] = ''
for i in range(len(data_df)):
    data_df['crytpo_related_keywords'][i] = get_crypto_related_keys(threshold,'spacy_nouns',embed_func,sim_func,
                                                                vocab,data_df['content'][i])

C:\Users\vivit\anaconda3\envs\StockAI37\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [136]:
data_df[41140:]

,id,content,crytpo_related_keywords
41140,371624,the catholic church didnt think that all books...,[catholic church]
41141,371625,clickbait title going to clickbait,[]
41142,371626,not following,[]
41143,371627,now your denzinger too,[]
41144,371628,the 95 theses were written by martin luther,[]
...,...,...,...
43481,374466,can find it in many places\nalso on santiment ...,"[santiment historical balances, selected time ..."
43482,374467,guys does anyone know if there is an applicati...,"[wallets, connection, transactions, wallets]"
43483,374468,any lobsters going to kyiv web3 hackathon sept...,[kyiv web3 hackathon]
43484,374469,whats funny is that no one complains about the...,[ethermine]


In [137]:
data_df.to_csv('C:\\Users\\vivit\\Downloads\\data_df_w_crypto_keyW1.csv')

In [146]:
# test example
threshold = 0.7
s1 = "to analyze blockchain security and find even better selfish mining techniques"
s2 =  "By the way, which do you think is the onchain analytics with the best user experience? Preferably the onesthat are self served and that everyone on the team can use"
s3 = "We're launching incentivized testnet on polygon at tokensoft today"
s_list = [s1,s2,s3]
embed_func = wordEmbedder("word sensitive").get_vectors
sim_func = keyMatcher("cosine").get_similarity

for s in s_list:
    s = clean(s)
    print(get_crypto_related_keys(threshold,'rake',embed_func,sim_func, vocab, s))


['analyze blockchain security']
['onchain analytics']
['tokensoft today', 'polygon']


In [ ]:
"""Advantages of this method:
* model-free
* will work for any given corpus
* works on a very small set of vocab words as it is purely algorithm based
* the embeddings are designed such that words that contain those occuring in the vocab as substrings
are also detected. For example, it is enough to have the word 'crypto' in order to detect words like 
'cryptoeconomics'. Or 'ether' to detect the word 'ethermine'.
Limitations:
* the embeddings don't work well when the keywords/phrases extracted are too long
* even though it doesn't use a model, the time taken is quite high. Not very space/time efficient.
* these limitations can be overcome by pruning the vocabulary thoroughly.
"""